In [3]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv

load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini")

In [4]:
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from typing import List

class ListaNumeros(BaseModel):
    numeros: List[int] = Field(description="Lista de números")

@tool("somar_lista", args_schema=ListaNumeros)
def somar_lista(numeros: list) -> int:
    """Soma uma lista de números"""
    soma = 0
    for numero in numeros:
        soma = soma + numero
    return soma

@tool("multiplicar_lista", args_schema=ListaNumeros)
def multiplicar_lista(numeros: list) -> int:
    """Multiplica uma lista de números"""
    produto = 1
    for numero in numeros:
        produto = produto * numero
    return produto


tools = [somar_lista, multiplicar_lista]

graph = create_react_agent(llm, tools=tools)

In [5]:
graph.invoke({"messages":"Quanto é 2043 vezes 17, mais 12?"})

{'messages': [HumanMessage(content='Quanto é 2043 vezes 17, mais 12?', additional_kwargs={}, response_metadata={}, id='d24836df-297e-4962-b30c-f6c5798561f9'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_z79RLjerhpp2yRm8WYawC8hm', 'function': {'arguments': '{"numeros": [2043, 17]}', 'name': 'multiplicar_lista'}, 'type': 'function'}, {'id': 'call_crVvB08E4hOKNEP6kZVrOxWK', 'function': {'arguments': '{"numeros": [2043, 12]}', 'name': 'somar_lista'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 91, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5129123b-de1d-4d76-9a7c-1f8afc0e3435-

In [6]:
grafo_bonito = (graph | (lambda x: x["messages"][-1].content))

print(grafo_bonito.invoke({"messages":"Quanto é 2043 vezes 17, mais 12?"}))

O resultado de 2043 vezes 17, mais 12, é 34.743.


In [8]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [9]:
from langchain import hub
from typing_extensions import TypedDict, Annotated
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str


class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]

query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}


def execute_query(state: State):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDataBaseTool(db=db)
    return {"result": execute_query_tool.invoke(state["query"])}

def generate_answer(state: State):
    """Answer question using retrieved information as context."""
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f'Question: {state["question"]}\n'
        f'SQL Query: {state["query"]}\n'
        f'SQL Result: {state["result"]}'
    )
    response = llm.invoke(prompt)
    return {"answer": response.content}

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")
system_message = prompt_template.format(dialect="SQLite", top_k=5)
system_message = system_message + "Always look for close names of values whenever a query returns empty. Never return empty handed until you are sure the information is not on the database.\n"
system_message = system_message + "Dont use LIMIT if the user wants all of information on the database.\n"
write_query({"question": "Quantos empregados existem no total?"})

/home/jpfassina/.cache/pypoetry/virtualenvs/cursolangchain-2sMNaNbo-py3.10/lib/python3.10/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/home/jpfassina/.cache/pypoetry/virtualenvs/cursolangchain-2sMNaNbo-py3.10/lib/python3.10/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


{'query': 'SELECT COUNT(*) as TotalEmployees FROM Employee;'}

In [10]:
execute_query({"query": "SELECT COUNT(EmployeeId) AS EmployeeCount FROM Employee;"})

/tmp/ipykernel_31483/3219298853.py:36: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  execute_query_tool = QuerySQLDataBaseTool(db=db)


{'result': '[(8,)]'}

In [11]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

agente = create_react_agent(llm, tools, state_modifier=system_message)

In [12]:
def answer_question(question:str):
    for step in agente.stream(
        {"messages": [{"role": "user", "content": question}]},
        stream_mode="values",
    ):
        step["messages"][-1].pretty_print()

In [13]:
answer_question("Quais são as tabelas que você tem acesso?")

================================ Human Message =================================

Quais são as tabelas que você tem acesso?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_kmjBXfsGSKB3lScVNYRr6KMp)
 Call ID: call_kmjBXfsGSKB3lScVNYRr6KMp
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================

As tabelas que tenho acesso são:

1. Album
2. Artist
3. Customer
4. Employee
5. Genre
6. Invoice
7. InvoiceLine
8. MediaType
9. Playlist
10. PlaylistTrack
11. Track


In [11]:
answer_question("Quais são todos os albums que existem na base?")

================================ Human Message =================================

Quais são todos os albums que existem na base?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_PKlHwQhRwRRf4uRDUCCt2Dxy)
 Call ID: call_PKlHwQhRwRRf4uRDUCCt2Dxy
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_Imi4a9Qkp0O7kNfqGdNAUxeq)
 Call ID: call_Imi4a9Qkp0O7kNfqGdNAUxeq
  Args:
    table_names: Album
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOR

In [12]:
answer_question("Quantas vendas o album big one teve?")

================================ Human Message =================================

Quantas vendas o album big one teve?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_8glSfuQxAv4xwEbuyGLtlsCN)
 Call ID: call_8glSfuQxAv4xwEbuyGLtlsCN
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_cM34UfuMaJtdVS6wDCbx2amc)
 Call ID: call_cM34UfuMaJtdVS6wDCbx2amc
  Args:
    table_names: Album
  sql_db_schema (call_C4AQCOcdnBNiOrU1YackFC2q)
 Call ID: call_C4AQCOcdnBNiOrU1YackFC2q
  Args:
    table_names: InvoiceLine
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "InvoiceLine" (
	"Inv

In [14]:
answer_question("Qual o vendedor mais velho de casa e qual é o seu album mais vendido?")

================================ Human Message =================================

Qual o vendedor mais velho de casa e qual é o seu album mais vendido?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_otP7OCXVWuQWWMXINnKP7XGT)
 Call ID: call_otP7OCXVWuQWWMXINnKP7XGT
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_e4tpcA1SdS2Ax5p7qvycbII7)
 Call ID: call_e4tpcA1SdS2Ax5p7qvycbII7
  Args:
    table_names: Employee
  sql_db_schema (call_cAmalX2WZPGRckfhZu1ygNeM)
 Call ID: call_cAmalX2WZPGRckfhZu1ygNeM
  Args:
    table_names: Album
  sql_db_schema (call_2SqBEXEl2NGDRP9dBBv1XZqx)
 Call ID: call_2SqBEXEl2NGDRP9dBBv1XZqx
  Args:
    table_n